https://pytorch.org/tutorials/intermediate/char_rnn_generation_tutorial.html

In [1]:
import pickle
with open("./names.pkl","rb") as file:
  names = pickle.load(file)

In [2]:
train_set = names[:int(len(names)*0.8)]
test_set = names[int(len(names)*0.8):]

In [3]:
languages_set = set()
for arr in [train_set,test_set]:
  for el in arr:
    languages_set.add(el[1])

In [4]:
languages_set=list(languages_set)
n_categories=len(languages_set)

In [5]:
len(train_set),len(test_set), len(names)

(16059, 4015, 20074)

In [13]:
import string
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)+1

# One-hot vector for category
def categoryTensor(category):
    li = languages_set.index(category)
    tensor = torch.zeros(1, n_categories)
    tensor[0][li] = 1
    return tensor

# One-hot matrix of first to last letters (not including EOS) for input
def inputTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li in range(len(line)):
        letter = line[li]
        tensor[li][0][all_letters.find(letter)] = 1
    return tensor

# ``LongTensor`` of second letter to end (EOS) for target
def targetTensor(line):
    letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
    letter_indexes.append(n_letters - 1) # EOS
    return torch.LongTensor(letter_indexes)

In [24]:
import torch
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        self.i2h = nn.Linear(n_categories + input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(n_categories + input_size + hidden_size, output_size)
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, category, input, hidden):
        input_combined = torch.cat((category, input, hidden), 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden, output), 1)
        output = self.o2o(output_combined)
        output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_letters)

In [26]:
lr = 0.0005 # If you set this too high, it might explode. If too low, it might not learn
from torch.optim import AdamW
optimizer = AdamW(rnn.parameters(), lr=lr)
criterion = nn.NLLLoss()
def train(category_tensor, input_line_tensor, target_line_tensor):
    target_line_tensor.unsqueeze_(-1)
    hidden = rnn.initHidden()

    optimizer.zero_grad()

    loss = torch.Tensor([0]) # you can also just simply use ``loss = 0``

    for i in range(input_line_tensor.size(0)):
        output, hidden = rnn(category_tensor, input_line_tensor[i], hidden)
        l = criterion(output, target_line_tensor[i])
        loss += l

    loss.backward()
    optimizer.step()

    return output, loss.item()

In [27]:
import time
import math
import random
n_iters = 200000
print_every = 5000
plot_every = 1000
from tqdm import tqdm
from statistics import mean

# Keep track of losses for plotting
current_loss = 0
all_losses = []

def randomTrainingExample():
    el = random.choice(train_set)
    line = el[0]
    category = categoryTensor(el[1])
    input_line_tensor = inputTensor(line)
    target_line_tensor = targetTensor(line)
    return category, input_line_tensor, target_line_tensor

# def test():
#     losses = []
#     with torch.no_grad():
#       for el in test_set:
#          line_tensor, category_tensor = lineToTensor(el[0]), torch.tensor([languages_set.index(el[1])], dtype=torch.long)
#          hidden = rnn.initHidden()
#          for i in range(line_tensor.size()[0]):
#              output, hidden = rnn(line_tensor[i], hidden)
#          loss = criterion(output, category_tensor)
#          losses.append(loss.item())
#     return mean(losses)


start = time.time()

for iter in tqdm(range(1, n_iters + 1)):
    category, input_line_tensor, target_line_tensor  = randomTrainingExample()
    output, loss = train(category, input_line_tensor, target_line_tensor)
    current_loss += loss

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        print("train_loss", all_losses[-1])
        # print("test_loss",test())
        current_loss = 0

  1%|          | 1022/200000 [00:05<21:21, 155.27it/s]

train_loss 21.020474509716035


  1%|          | 2037/200000 [00:09<14:19, 230.25it/s]

train_loss 18.12153672504425


  2%|▏         | 3041/200000 [00:14<14:03, 233.63it/s]

...

train_loss 16.104493117570875


100%|█████████▉| 199015/200000 [15:55<00:06, 152.73it/s]

train_loss 15.389915365219116


100%|██████████| 200000/200000 [16:00<00:00, 208.24it/s]

train_loss 15.848558926701546


In [32]:
max_length = 20

# Sample from a category and starting letter
def sample(category, start_letter='A'):
    with torch.no_grad():  # no need to track history in sampling
        category_tensor = categoryTensor(category)
        input = inputTensor(start_letter)
        hidden = rnn.initHidden()

        output_name = start_letter

        for i in range(max_length):
            output, hidden = rnn(category_tensor, input[0], hidden)
            topv, topi = output.topk(1)
            topi = topi[0][0]
            if topi == n_letters - 1:
                break
            else:
                letter = all_letters[topi]
                output_name += letter
            input = inputTensor(letter)

        return output_name

# Get multiple samples from one category and multiple starting letters
def samples(category, start_letters='ABC'):
    for start_letter in start_letters:
        print(sample(category, start_letter))

samples('Russian', 'RUS')

samples('German', 'GER')

samples('Spanish', 'SPA')

samples('Chinese', 'CHI')

Rovich
Ushanovich
Sharoff
Gran
Eler
Rosher
Salein
Para
Abara
Cha
Han
Ina
